### 套件安裝指令

1. pip install git+https://github.com/APCLab/jieba-tw.git
2. pip install scikit-learn
3. pip install torch transformers
4. pip install gensim
5. pip install scipy==1.10.1  

In [1]:
import pandas as pd
from zhon.hanzi import punctuation
import re

In [2]:
def full_to_half(s):
    # 將字符串 s 中的全形字符和標點符號轉換為半形。
    n = []
    for char in s:
        code = ord(char)
        if code == 0x3000:  # 全形空格直接轉換
            code = 32
        elif 0xFF01 <= code <= 0xFF5E:  # 全形字符（除空格）轉換成半形
            code -= 0xFEE0
        n.append(chr(code))
    return ''.join(n)

In [3]:
with open('./Data/stopwords_tc.txt', encoding='utf-8', mode='r') as f:
    stop_words = []
    for l in f:
        stop_words.append(l.strip())

In [4]:
ptt_food_post_df = pd.read_csv('./Data/Ptt/ptt_food_post_list.csv', index_col='Pid')
ptt_food_post_df

,title,link,date,content
Pid,,,,
0,[食記] 台南 來一片。一周只賣三天的美味披蕯店,https://www.ptt.cc/bbs/Food/M.1669640419.A.EFD...,2022/11/28,餐廳名稱：來一片消費時間：2022年/10月電話：0905-539108地址：台南市仁德區二...
1,[食記] 台北 辰壽司割烹~大閘蟹宴,https://www.ptt.cc/bbs/Food/M.1669642084.A.CE7...,2022/11/28,餐廳名稱：辰壽司割烹 消費時間：2022年/11月 地址：台北市松山區敦化北路...
2,[食記] 新北 金大鋤壽喜燒烤鍋物新莊店下午茶,https://www.ptt.cc/bbs/Food/M.1669646872.A.E61...,2022/11/28,鍋物最前線--金大鋤壽喜燒烤鍋物新莊店下午茶消費日期：2022 年 11 月有圖有真相圖文並...
3,[食記] 新北林口｜林口三井的點點心,https://www.ptt.cc/bbs/Food/M.1669680859.A.700...,2022/11/29,餐廳名稱：林口三井OUTLET|放鬆心情 來吃港式的點點心 消費時間：2022/1...
4,[食記] 台北松山 夯・魯肉飯 原本的五分埔魯肉飯,https://www.ptt.cc/bbs/Food/M.1669706692.A.EFC...,2022/11/29,餐廳名稱：夯・魯肉飯 消費時間：2022年/10月110台北市信義區松山路152號...
...,...,...,...,...
1002,[食記] 台北 中山區 欣葉台菜創始店,https://www.ptt.cc/bbs/Food/M.1664473643.A.7CA...,2022/09/30,餐廳名稱： 欣葉台菜創始店消費時間：2022年9月地址：台北市中山區雙城街34之1號營業時間...
1003,[食記] 台北信義-寓所咖啡Utroo cafe-咖啡廳,https://www.ptt.cc/bbs/Food/M.1664505703.A.CC6...,2022/09/30,餐廳名稱 : 寓所咖啡 Utroo café消費時間 : 2022年/8月地址 : ...
1004,[食記] 屏東潮州-牛大福。屏東牛肉料理,https://www.ptt.cc/bbs/Food/M.1664507458.A.84A...,2022/09/30,餐廳名稱：屏東。潮州》牛大福。屏東牛肉料理 消費時間：2022年/8月 地址：...


In [5]:
gpt_food_post_df = pd.read_csv('./Data/ChatGPT/chatgpt_generated_articles.csv')
gpt_food_post_df

,title,content,prompt_system,prompt_user
0,[食記] 台北 辰壽司割烹~大閘蟹宴,[食記] 台北 辰壽司割烹~大閘蟹宴\n\n- 餐廳名稱：辰壽司割烹\n- 消費時間：202...,你是美食部落客,發表食記，先提供餐廳名稱、消費時間、地址、電話、營業時間，消費時間隨機選2022年以前的月份...
1,[食記] 新北 金大鋤壽喜燒烤鍋物新莊店下午茶,[食記] 新北 金大鋤壽喜燒烤鍋物新莊店下午茶\n\n◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢◤◢...,你是美食熱愛者,寫一篇「[食記] 新北 金大鋤壽喜燒烤鍋物新莊店下午茶」為標題的文章，風格：PTT發文風格，...
2,[食記] 新北林口｜林口三井的點點心,[食記] 新北林口｜林口三井的點點心\n\n大家好，這裡是小葵，今天來跟大家分享我最近造訪的...,你是美食部落客,以PTT美食版發文風格，寫一篇「[食記] 新北林口｜林口三井的點點心」為標題的文章，要包含以...
3,[食記] 台北松山 夯・魯肉飯 原本的五分埔魯肉飯,[食記] 台北松山 夯・魯肉飯 原本的五分埔魯肉飯\n\n各位美食好手們，今日小編要介紹的是...,你是美食部落客,以PTT美食版發文風格，寫一篇「[食記] 台北松山 夯・魯肉飯 原本的五分埔魯肉飯」為標題的...
4,[食記] 桃園。麵屋虎千代,[食記] 桃園。麵屋虎千代\n\n餐廳名稱：麵屋虎千代\n地址：桃園市中壢區中園路123號\...,你是美食評論家,寫一篇「[食記] 桃園。麵屋虎千代」為標題的文章，風格：PTT發文風格，要有餐廳名稱、地址、...
...,...,...,...,...
1001,[食記] 屏東潮州-牛大福。屏東牛肉料理,[食記] 屏東潮州-牛大福。屏東牛肉料理\n\n大家好，這次要跟大家分享在屏東潮州區一家美味...,你是美食熱愛者,以PTT美食版發文風格，寫一篇「[食記] 屏東潮州-牛大福。屏東牛肉料理」為標題的文章，要包...
1002,[食記] 個人評比 肉次方VS夯下去,[食記] 個人評比 肉次方VS夯下去\n\n餐廳名稱：肉次方\n消費時間：2022/09/1...,你是美食熱愛者,在美食論壇發表食記，先提供餐廳名稱、消費時間(2022/11以前)、地址、電話、營業時間，引...
1003,[食記] 桃園龍潭 糧園茶藝客家小館,[食記] 桃園龍潭 糧園茶藝客家小館\n\n餐廳名稱：糧園茶藝客家小館\n消費時間：2022...,你是美食部落客,在美食論壇發表食記，先提供餐廳名稱、消費時間(2022/11以前)、地址、電話、營業時間，引...
1004,[食記] 台北信義安和 吉可頌丹麥專賣店,[食記] 台北信義安和 吉可頌丹麥專賣店\n\n消費時間：2021年6月\n地址：台北市信義...,你是美食熱愛者,發表食記，先提供餐廳名稱、消費時間、地址、電話、營業時間，消費時間隨機選2022年以前的月份...


### Stopwords Removal  

In [6]:
def remove_urls_and_phones(text):
    """
    移除文字中的網址和電話號碼。
    """
    # 正則表達式匹配網址
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    text = re.sub(url_pattern, '', text)

    # 正則表達式匹配電話號碼（適用於多種常見格式）
    phone_pattern = r'(\d{2,4}[-.\s]??\d{3,4}[-.\s]??\d{3,4}|\(\d{2,4}\)\s*\d{3,4}[-.\s]??\d{3,4}|\d{10,11})'
    text = re.sub(phone_pattern, '', text)

    return text

In [7]:
def remove_english(text):
    """
    移除文字中的所有英文字符。
    """
    # 正則表達式匹配所有英文字母和英文單詞
    pattern = r'[A-Za-z]+'
    cleaned_text = re.sub(pattern, '', text)

    return cleaned_text

In [8]:
def remove_emojis(text):
    emoji_pattern = re.compile(
        "[" 
        "\U0001F600-\U0001F64F"  # 表情符號
        "\U0001F300-\U0001F5FF"  # 符號和圖案
        "\U0001F680-\U0001F6FF"  # 交通和符號
        "\U0001F700-\U0001F77F"  # 藝術符號
        "]+",
        flags=re.UNICODE,
    )
    return emoji_pattern.sub(r'', text)

In [9]:
def load_dictionary(file_path):
    with open(file_path, encoding='utf-8', mode='r') as f:
        dictionary = [l.strip() for l in f]
    return dictionary

In [10]:
def remove_stopwords(text, stopwords):
    # 使用列表推導式過濾掉停用詞
    filtered_words = remove_urls_and_phones(text)
    filtered_words = remove_english(filtered_words)
    filtered_words = remove_emojis(filtered_words)
    filtered_words = "".join(c for c in filtered_words if c not in ('；','，','。','！','：','「','」','…','、','？','【','】','.',':','?',';','!','~','`','+','-','<','>','/','[',']','{','}',"'",'"','\\', ' ', '‧','・','◢','◤','\n','★','☆','◆','◇','◎','○','●','◐','◑','▲','▼','△','▽','◢','◣','◥','◤','▷','◁','▶','◀','♠','♣','♥','♦','♨','⊙','⊕','▨','▧','▦','▥','▤','▣','▢','□','■'))
    filtered_words = "".join(word for word in filtered_words if word not in stopwords)
    filtered_words = filtered_words.replace(u'\u3000', u' ')
    # 將過濾後的單詞列表重新組合成字符串
    return filtered_words

### Test

In [11]:
ptt_contents = ptt_food_post_df.loc[:, 'content']
gpt_contents = gpt_food_post_df.loc[:, 'content']

In [12]:
stop_words = load_dictionary('./Data/stopwords_tc.txt')
ptt_remove_stopword_contents = []
gpt_remove_stopword_contents = []
for c in ptt_contents:
    ptt_remove_stopword_contents.append(remove_stopwords(c, stop_words))

for c in gpt_contents:
    gpt_remove_stopword_contents.append(remove_stopwords(c, stop_words))

# 添加標記並合併 DataFrame
ptt_contents_df = pd.DataFrame(ptt_remove_stopword_contents, columns=['remove_stopword_content'])
ptt_contents_df['source'] = 'ptt'

gpt_contents_df = pd.DataFrame(gpt_remove_stopword_contents, columns=['remove_stopword_content'])
gpt_contents_df['source'] = 'gpt'

combined_df = pd.concat([ptt_contents_df, gpt_contents_df], ignore_index=True)
combined_df

,remove_stopword_content,source
0,餐廳名稱片消費時間年月電話址台南市仁德區空路號營業時間日圖文版片位台南空眷村家低調披蕯店原平...,ptt
1,餐廳名稱辰壽司割烹消費時間年月址台北市松山區敦化北路號營業時間平價位圖文月底結束前朋友揪局覺...,ptt
2,鍋物前線金鋤壽喜燒烤鍋物新莊店午茶消費日期年月圖真相圖文茂版點新北市新莊區幸福路號電話價位動...,ptt
3,餐廳名稱林口井放鬆心情吃港式點點心消費時間址新北市林口區文化路段號樓電話營業時間週週週週週日...,ptt
4,餐廳名稱夯魯肉飯消費時間年月台北市信義區松山路號圖文網誌分數低評破位數裝潢實文青說真點詹記感...,ptt
...,...,...
2008,食記屏東潮州牛福屏東牛肉料理家次家分享屏東潮州區家美味牛肉料理餐廳牛福裡品嚐道美味牛肉料理回...,gpt
2009,食記評肉次夯餐廳名稱肉次消費時間址台北市安區復興南路段號電話營業時間週週日進入肉次空間刻溫暖...,gpt
2010,食記桃園龍潭糧園茶藝客家館餐廳名稱糧園茶藝客家館消費時間址桃園市龍潭區路號電話營業時間週週日...,gpt
2011,食記台北信義安吉頌丹麥專賣店消費時間年月址台北市信義區松仁路號電話營業時間週週日天台北信義區...,gpt


### Vectorlize

In [13]:
remove_stopword_contents = combined_df['remove_stopword_content']
remove_stopword_contents_list = remove_stopword_contents.to_list()
remove_stopword_contents_list

['餐廳名稱片消費時間年月電話址台南市仁德區空路號營業時間日圖文版片位台南空眷村家低調披蕯店原平日賣鍋假日賣披蕯現周營業天主披蕯分享文章中發現片主廚台中鍋牛台北服務主廚創意披蕯獎項片店址位國宅區華麗裝潢座位般餐館樣桌數想餐建議訂位市區生意真錯餐點容周調整說片價格真平實主外開胃點義利麵蝦膏濃湯附片麵包明蝦湯頭真蝦味濃喝松露炸薯條薯修炸微酥灑松露提味瑪格利特披薩說樣元真宜桌聞香氣餅皮微酥愈嚼香愈香搭蕃茄司羅勒酸甜風味基款披蕯真超美味坡里簡炸海鮮蝦子丁香魚魚肉片中卷炸微酥油膩足茄汁甜椒培根義利麵麵條吃相彈嚼勁茄汁甜感度十足榛果巧克力醬炸披薩條披蕯皮炸酥淋巧克力醬披蕯皮炸酥軟吃甜辣樣淋巧克力醬甜甜整盤吃完熱量爆表攝影旅行拈花惹草',
 '餐廳名稱辰壽司割烹消費時間年月址台北市松山區敦化北路號營業時間平價位圖文月底結束前朋友揪局覺價位主吃費工爽度謙主理閘蟹料理錯蟹湯蒸蛋蟹肉膏握紫蘇炸蟹道印象特兩月前吃複習舊吃正餐觀點配餐反倒潑元美味度輸次品嚐期間限特色菜心情吃蟹宴道水海膽蟹肉入藥膳元素開場溫協調綠蘆筍冷湯蟹肉道融合感差少許爽口印象深中間穿插握壽司挺尾熊明蝦甜美富彈脆度接蟹湯茶碗蒸日愛熱氣蟹黃膏泥風味擴散蟹肉芡汁量夠鮮秋刀次佳吃頭尾時期閘蟹肉膏握壽司暴力貫樣閘蟹紫蘇揚勝次知回事彧割烹烤飯香吃作填肚子赤味噌加膏黃吃法趣味道變溫醇吃法喜聽兩枸杞藥膳凍馬糞水海膽閘蟹肉長崎軟絲握鹽花天鰤寒作里辣椒閘蟹肉蟹油荷蘭綠蘆筍冷湯昆布高湯生筋子星米米香熊明蝦握北海道根室添加海膽軍艦銀杏年糕烏魚子蟹湯生卵蒸蛋蟹肉芡汁魚子醬蟹黃膏秋刀握閘蟹肉膏握少許鎮江醋閘蟹膏手捲閘蟹紫蘇天婦羅椴木香菇鹽昆布台東旗魚握熟成日魚肉纖維細含脂宮城黑鮪蛇腹熟成日蛇腹貫招貫軟筋口感優黑扇蝦奶油燒風乾鱘魚子蟹黃膏甲羅燒烤飯黑松露片聞香吃玉子燒赤味噌湯閘蟹膏放進攪拌紅味噌變溫點妙想法黑糖薑蜜蘋果生薑雪酪甜點吃刻意薑祛寒',
 '鍋物前線金鋤壽喜燒烤鍋物新莊店午茶消費日期年月圖真相圖文茂版點新北市新莊區幸福路號電話價位動目前止平日優惠價元需加成服務費環境位食尚廣場新莊幸福館樓點捷運算段附停車算場館提供付費停車場果汽車找停車位位置走進樓中庭空間寬敞店環境分左右兩側餐區中間區域主助區取餐點店寬敞般動線算太擁擠餐尖峰期間家擠擁擠候情形天平日午茶時段整場組左右客餐相悠閒選座位相錯食尚廣場新莊幸福館店面文宣板櫃台餐環境餐點餐桌餐具調味料區

In [14]:
import jieba
# 使用 Jieba 進行切詞
def jieba_cut(text):
    return ' '.join(jieba.cut(text))

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

def tfidf_vectorlize(posts):

    # 將每篇文章進行切詞
    cut_posts = [jieba_cut(post) for post in posts]

    # 初始化 TfidfVectorizer，這裡可以調整參數
    vectorizer = TfidfVectorizer()

    # 將切詞後的文章內容轉換為 TF-IDF 矩陣
    tfidf_matrix = vectorizer.fit_transform(cut_posts)

    # 獲取特徵名稱（詞彙）
    feature_names = vectorizer.get_feature_names_out()

    # 將 TF-IDF 矩陣轉換為 DataFrame
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), index=range(len(posts)), columns=feature_names)

    return tfidf_df

In [16]:
# 提取文章內容並進行 TF-IDF 向量化
tfidf_df = tfidf_vectorlize(combined_df['remove_stopword_content'])

# 將標記欄位添加回 TF-IDF DataFrame 中
tfidf_df['source'] = combined_df['source']
tfidf_df.to_csv('./Data/all_tfidf_vector.csv')

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/rh/7g0n3djn3wj8dlnjv7plmzcr0000gn/T/jieba.cache
Loading model cost 0.145 seconds.
Prefix dict has been built succesfully.
